In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![View on GitHub][github-badge]][github-keras-v1] [![Open In Colab][colab-badge]][colab-keras-v1] [![Open in Binder][binder-badge]][binder-keras-v1]

[github-badge]: https://img.shields.io/badge/View-on%20GitHub-blue?logo=GitHub
[colab-badge]: https://colab.research.google.com/assets/colab-badge.svg
[binder-badge]: https://static.mybinder.org/badge_logo.svg

[github-keras-v1]: LeNet_v1_basic_impl_in_Keras.ipynb
[colab-keras-v1]: https://colab.research.google.com/github/mbrukman/reimplementing-ml-papers/blob/main/lenet/LeNet_v1_basic_impl_in_Keras.ipynb
[binder-keras-v1]: https://mybinder.org/v2/gh/mbrukman/reimplementing-ml-papers/main?filepath=lenet/LeNet_v1_basic_impl_in_Keras.ipynb

In [ ]:
from tensorflow import keras
from keras import Input, Sequential
from keras.layers import Activation, AveragePooling2D, MaxPooling2D, Conv2D, Dense, Flatten

We will start with a very simple approximation of the network described in the paper and evolve it over time to more closely match the paper.

For one, there isn't a built-in Keras layer that matches the subsampling layer in the paper: neither `AveragePooling2D` nor `MaxPooling2D` have any trainable parameters, but the subsampling layer described in the paper does, so this is already one difference.

The activation function is a more complex function than the ReLU we're using here, but it's a reasonable approximation, and even with these changes, we get quite a good accuracy on both training and test sets.

In [ ]:
tanh = keras.activations.tanh
softmax = keras.activations.softmax

# Define the model architecture.
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation=tanh, name='C1'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'),
    Activation(tanh, name='S2_act'),
    Conv2D(filters=16, kernel_size=(5, 5), activation=tanh, name='C3'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'),
    Activation(tanh, name='S4_act'),
    Flatten(),
    Dense(120, activation=tanh, name='C5'),
    Dense(84, activation=tanh, name='F6'),
    Dense(10, activation=softmax, name='Output'),
], name='LeNet-5')

model.summary()

Model: "LeNet-5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C1 (Conv2D)                 (None, 28, 28, 6)         156       
                                                                 
 S2 (MaxPooling2D)           (None, 14, 14, 6)         0         
                                                                 
 S2_act (Activation)         (None, 14, 14, 6)         0         
                                                                 
 C3 (Conv2D)                 (None, 10, 10, 16)        2416      
                                                                 
 S4 (MaxPooling2D)           (None, 5, 5, 16)          0         
                                                                 
 S4_act (Activation)         (None, 5, 5, 16)          0         
                                                                 
 flatten (Flatten)           (None, 400)               0   

In [ ]:
# Compile the model with optimizer and loss function.
opt = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

For details on the MNIST dataset including a data exploration, see [MNIST directory in my repo](https://github.com/mbrukman/reimplementing-ml-papers/tree/main/datasets/mnist).

Here, we will import a shared library to process the MNIST dataset into the format that we need to use below for model training and testing.

In [ ]:
%%bash

# Download our library for processing MNIST dataset.
if ! [ -f "mnist.py" ]; then
  curl -sO https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/datasets/mnist/mnist.py
fi

In [ ]:
%%capture --no-stderr

import mnist

# This will download the MNIST dataset via the Keras library which outputs data
# to stdout, so we silence it above to avoid extraneous output.
mnist_data = mnist.MNIST()

In [ ]:
# Train the model.
#
# In this notebook, we scale the input into the range [0.0, 1.0] and convert the
# labels y to a categorical (one-hot) encoding from the default numeric values.
#
# For consistency, we use the same transformations for the test dataset below.
model.fit(mnist_data.x_train_scale_0_1(),
          mnist_data.y_train_categorical(),
          epochs=20)

Epoch 1/20
1875/1875 [==============================] - 14s 3ms/step - loss: 0.1614 - accuracy: 0.9517
Epoch 2/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0543 - accuracy: 0.9833
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0369 - accuracy: 0.9886
Epoch 4/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0269 - accuracy: 0.9921
Epoch 5/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0203 - accuracy: 0.9937
Epoch 6/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0174 - accuracy: 0.9946
Epoch 7/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0133 - accuracy: 0.9957
Epoch 8/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0128 - accuracy: 0.9959
Epoch 9/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0092 - accuracy: 0.9971
Epoch 10/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.00

In [ ]:
# Evaluate the model.
#
# Note that we use the same input range scaling and label encoding as above.
model.evaluate(mnist_data.x_test_scale_0_1(), mnist_data.y_test_categorical())

313/313 [==============================] - 1s 3ms/step - loss: 0.0565 - accuracy: 0.9862


[0.056457001715898514, 0.9861999750137329]